In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier  # Example model
from sklearn.metrics import classification_report

# Step 1: Combine data from all sensors
dataframes = []  # List to hold individual sensor DataFrames
for sensor_id in range(1, 64):
    df_sensor = pd.read_csv(f'sensor_{sensor_id}.csv')  # Load each sensor's data
    df_sensor['sensor_id'] = sensor_id  # Add sensor identifier
    dataframes.append(df_sensor)

combined_df = pd.concat(dataframes, ignore_index=True)

# Step 2: Feature engineering
# Assuming features are already engineered in each sensor DataFrame

# Step 3: Train-test split
train_size = int(len(combined_df) * 0.8)
train_df = combined_df[:train_size]
test_df = combined_df[train_size:]

# Alternatively, use time-based cross-validation
# tscv = TimeSeriesSplit(n_splits=5)

# Step 4: Model training
X_train = train_df.drop(['anomaly_label'], axis=1)  # Drop label column
y_train = train_df['anomaly_label']
X_test = test_df.drop(['anomaly_label'], axis=1)
y_test = test_df['anomaly_label']

model = RandomForestClassifier()
model.fit(X_train, y_train)

# Step 5: Model evaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Step 6: Sensor-specific performance
for sensor_id in combined_df['sensor_id'].unique():
    sensor_test_df = test_df[test_df['sensor_id'] == sensor_id]
    X_sensor_test = sensor_test_df.drop(['anomaly_label'], axis=1)
    y_sensor_test = sensor_test_df['anomaly_label']
    
    y_sensor_pred = model.predict(X_sensor_test)
    print(f"Performance for sensor {sensor_id}:")
    print(classification_report(y_sensor_test, y_sensor_pred))
